In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")


## Create SparkContext, SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext()

In [2]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

In [5]:
data = sc.textFile("/user/thomasj/UberUseCase/HistoricalData/")

In [6]:
data.count()

3296451

In [7]:
data.take(10)

[u'"12/1/2016 0:03:00",40.7586,-73.9706,"B02512"',
 u'"12/1/2016 0:05:00",40.7605,-73.9994,"B02512"',
 u'"12/1/2016 0:06:00",40.732,-73.9999,"B02512"',
 u'"12/1/2016 0:09:00",40.7635,-73.9793,"B02512"',
 u'"12/1/2016 0:20:00",40.7204,-74.0047,"B02512"',
 u'"12/1/2016 0:35:00",40.7487,-73.9869,"B02512"',
 u'"12/1/2016 0:57:00",40.7444,-73.9961,"B02512"',
 u'"12/1/2016 0:58:00",40.7132,-73.9492,"B02512"',
 u'"12/1/2016 1:04:00",40.759,-73.973,"B02512"',
 u'"12/1/2016 1:08:00",40.7601,-73.9823,"B02512"']

In [8]:
def convertDataFloat(line):
    return array([float(line[1]),float(line[2])])

In [9]:
fea_data = data.map(lambda data:data.split(','))
parsedData = fea_data.map(lambda line : convertDataFloat(line))

In [10]:
parsedData.take(5)

[array([ 40.7586, -73.9706]),
 array([ 40.7605, -73.9994]),
 array([ 40.732 , -73.9999]),
 array([ 40.7635, -73.9793]),
 array([ 40.7204, -74.0047])]

In [11]:
clusters = KMeans.train(parsedData,8, maxIterations=10,runs=10, initializationMode="random")

/usr/hdp/current/spark2-client/python/lib/pyspark.zip/pyspark/mllib/clustering.py:347: UserWarning: The param `runs` has no effect since Spark 2.0.0.


In [12]:
clusters.centers

[array([ 40.71226754, -73.94425417]),
 array([ 40.78850748, -73.95490255]),
 array([ 40.66815658, -73.9793437 ]),
 array([ 40.75710264, -73.98132872]),
 array([ 40.66535382, -73.75897644]),
 array([ 40.72750868, -73.9996304 ]),
 array([ 40.69888243, -74.20316156]),
 array([ 40.79238175, -73.86029026])]

In [13]:
def wsssError(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [14]:
WSSSE = parsedData.map(lambda point: wsssError(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 64148.4592883


In [15]:
#clusters.predict(array([40.7204,-74.0047]))

In [16]:
#sqrt(sum((array([40.7204,-74.0047]) - array([ 40.71743048, -74.002436  ])) ** 2))

In [17]:
#40.7204 - 40.71743048

In [18]:
#-74.0047 - -74.002436

In [19]:
clusters.save(sc, "/user/thomasj/UberUseCase/Model/kmeanModel")